# Generation of transmission eigenchannels and plots of these using cube #

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sisl
import glob as glob

## 0. Preparations ##

In [ ]:
import py3Dmol

def PlotXYZ(xyzfile, width=800, height=200, rotation=90, zoom=1):
    xyzstr = open(xyzfile, 'r').read()
    xyzview = py3Dmol.view(width=width, height=height)
    xyzview.addModel(xyzstr, 'xyz')
    xyzview.setStyle({'sphere': {'colorscheme': 'Jmol', 'scale': 0.3},
                      'stick': {'colorscheme': 'Jmol', 'radius': 0.2}})
    xyzview.rotate(rotation)
    xyzview.zoomTo()
    xyzview.zoom(zoom)
    xyzview.show()
    
def PlotCube(cubefiles, isoval, width=800, height=200, rotation=90, zoom=1):
    if not isinstance(cubefiles, list):
        cubefiles = [cubefiles]
    geom = sisl.get_sile(cubefiles[0]).read_geometry()
    geom.write('tmp.xyz')
    xyzstr = open('tmp.xyz', 'r').read()
    xyzview = py3Dmol.view(width=width, height=height)
    xyzview.addModel(xyzstr, 'xyz')
    xyzview.setStyle({'sphere': {'colorscheme': 'Jmol', 'scale': 0.3},
                      'stick': {'colorscheme': 'Jmol', 'radius': 0.2}})
    color = ["red", "blue", "green", "yellow"]
    for i, cube in enumerate(cubefiles):
        voldata = 'some line which is needed' + open(cube, 'r').read()
        xyzview.addVolumetricData(voldata, 'cube', 
                                  {'isoval': -isoval, 'color': color[2 * i % len(color)], 'opacity': 0.8})
        xyzview.addVolumetricData(voldata, 'cube',
                                  {'isoval': isoval, 'color': color[(2 * i + 1) % len(color)], 'opacity': 0.8})
    xyzview.rotate(rotation)
    xyzview.zoomTo()
    xyzview.zoom(zoom)
    xyzview.show()

## 1. System setup ##

We will use the tutorial `IN_01` on inelastic transport with a CO between Au chains:

In [ ]:
RUNdir = '../IN_01'
Device = RUNdir + '/TSrun/RUN.fdf'
ElecL = RUNdir + '/ELEC/RUN.fdf'
ElecR = ElecL

In [ ]:
Ham = sisl.get_sile(Device).read_hamiltonian()
no = Ham.no
sc = Ham.sc
Ham.set_nsc(c=1) # remove periodic boundaries along transport direction ("C" or z)

Pick an energy

In [ ]:
eta = 1e-6j # imaginary part in device and electrodes
En = -0.13 + eta

We use `sisl` to generate left and right self-energies

In [ ]:
HamL = sisl.get_sile(ElecL).read_hamiltonian()
SFEL = sisl.RecursiveSI(HamL, "-C", eta=eta).self_energy(En)
GamL = 1j * (SFEL - SFEL.T.conj())
nL = len(GamL)
iL = slice(0,nL)

In [ ]:
HamR = sisl.get_sile(ElecR).read_hamiltonian()
SFER = sisl.RecursiveSI(HamR, "+C", eta=eta).self_energy(En)
GamR = 1j * (SFER - SFER.T.conj())
nR = len(GamR)
iR = slice(no - nR, no)

Here we set up the the inverse GF and invert it: Note the `UseBulk` option (similar option can be found in `tbtrans` and `transiesta`)

In [ ]:
# Calculate GF
UseBulk = True
invG = Ham.Sk(format="array") * En - Ham.Hk(format="array")

if(UseBulk):
    HamL.set_nsc([None, None, 1])
    HamR.set_nsc([None, None, 1])
    invG[iL, iL] = HamL.Sk(format="array") * En - HamL.Hk(format="array") - SFEL
    invG[iR, iR] = HamR.Sk(format="array") * En - HamR.Hk(format="array") - SFER
else:
    invG[iL, iL] -= SFEL
    invG[iR, iR] -= SFER

G = np.linalg.inv(invG)

Left (${\bf A}_L$) and Right (${\bf A}_R$) spectral functions

In [ ]:
AL = G[:, iL] @ GamL @ (G[:, iL]).T.conj()
AR = G[:, iR] @ GamR @ (G[:, iR]).T.conj()

Calculate transmission

In [ ]:
Transmission = np.trace(AR[:, iL] @ GamL).real
print(Transmission)

Diagonalize  ${\bf A}_L$ and ${\bf A}_R$ to get the basis of left/right scattering states in the device region.

Note that we only pick those with a non-negligble DOS in the device region and that the dimension of the device region is $\le$ than the number of scattering states. We will in general have a different number of left and right scattering states.

In [ ]:
dos_tolerance = 0.00001
eig, vec = np.linalg.eigh(AL)
aeigL = eig[eig > dos_tolerance]
avecL = vec[:, eig > dos_tolerance]
avecLnorm = np.sqrt(aeigL / (2 * np.pi))[np.newaxis, :] * avecL

eig, vec = np.linalg.eigh(AR)
aeigR = eig[eig > dos_tolerance]
avecR = vec[:, eig > dos_tolerance]
avecRnorm = np.sqrt(aeigR / (2 * np.pi))[np.newaxis, :] * avecR

## 2. Left-to-Right eigenchannels ##

In [ ]:
T_RL = 2 * np.pi * avecLnorm[iR, :].T.conj() @ GamR @ avecLnorm[iR, :] # Transmission matrix
T_RLeig, T_RLvec = np.linalg.eigh(T_RL)
T_RLeig, T_RLvec = T_RLeig[::-1], T_RLvec[:, ::-1] 
Psi_RL =  avecLnorm @ T_RLvec / (2 * np.pi) ** 0.5

Eigenchannel transmissions, $\tau_n$

In [ ]:
T_RLeig[0:4]

Generate channels, $\Psi_n$, on a real-space grid

In [ ]:
maxeig = 3 # just plot the first two channels
CenterCell = np.sum(sc.cell, axis=0) / 2
CenterCell[2] = 0

for ieig in range(maxeig):
    wf = Psi_RL[:, ieig]
    # we scale the max amplitude to 1 and choose the phase:
    max_amp = -1.0
    phase = 1.0 + 0.0j
    for y in wf:
        if abs(y) > max_amp:
            max_amp = abs(y)
            phase = y / max_amp
    wf /= phase
    grid = sisl.Grid(0.2, sc=sc, bc=0, dtype=np.complex128, geometry=Ham.geometry.move(CenterCell))
    sisl.physics.electron.wavefunction(wf, grid=grid, eta=True)
    grid.write(f"EC_{ieig}_Re.cube", imag=False)
    grid.write(f"EC_{ieig}_Im.cube", imag=True)    

In [ ]:
cubes = glob.glob("*Re.cube")
cubes.sort()
for cube in cubes:
    PlotCube([cube, cube.replace("Re", "Im")], 0.008, zoom=3)

We can also compare with the corresponding states computed with the original `EigenChannels` script:

In [ ]:
cubes = glob.glob("../IN_01/ECrun/*Re.cube")
cubes.sort()
for cube in cubes:
    PlotCube([cube, cube.replace("Re", "Im")], 0.006, zoom=3)

## Exercise ##

* Try to generate the Right-to-Left eigenchannels, i.e., the electron waves incoming from the right electrode: Note how they differ from the Left-to-Right eigenchannels.